###### Objective: Create process to conduct market basket type of analysis among products

###### "Baskets" will be two services used together (not larger combinations 3+). Goal is to identify how usage of one service increases the likelihood of using another service and the financial impact

### Imports

In [4]:
'Importing libraries'
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
'Apriori libraries'
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
'CSV import'
import csv

### Dataset

In [5]:
#Fetch data from csv file
df = pd.read_excel('C:\dimensions\Account Basket\Spend Portfolio.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1087 entries, 0 to 1086
Columns: 143 entries, ACCOUNTID to Service 140
dtypes: float64(133), int64(8), object(2)
memory usage: 1.2+ MB


In [6]:
df

,ACCOUNTID,SpendSegment,MONTHID,Service 1,Service 2,Service 3,Service 4,Service 5,Service 6,Service 7,...,Service 131,Service 132,Service 133,Service 134,Service 135,Service 136,Service 137,Service 138,Service 139,Service 140
0,076e4d78-2704-4463-bb18-326fdd51411d,3: $10K to <$100K,202202,0.459860,0,0.0,0.0,0.0,0.0,0.0,...,3.742104,0.0,0.000000,0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0e9ebc97-7ea8-4fa8-b3b5-0c06d25fe63b,3: $10K to <$100K,202202,0.000022,0,0.0,0.0,0.0,0.0,0.0,...,5.110449,134.4,0.000000,0,0.0,0.0,0.000000,0.0,0.0,0.0
2,180f0ffb-2ee9-4b0a-925c-7db258e3fabd,3: $10K to <$100K,202202,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,5.358268,201.6,0.000000,0,0.0,0.0,6692.018671,0.0,0.0,0.0
3,18322cec-f95e-4e0d-ab23-938e74519211,3: $10K to <$100K,202202,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,91.914175,0,0.0,0.0,0.000000,0.0,0.0,0.0
4,1c137916-8b2e-4dfd-8615-c66e6a1b4700,1: >$0 to <1K,202202,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,f2bd5d43-145f-4a72-8b73-1e36ea27ba15,2: $1K to <$10K,202202,0.003099,0,0.0,0.0,0.0,0.0,0.0,...,1.151096,131.1,0.000000,0,0.0,0.0,22.000000,0.0,0.0,0.0
1083,f64dc09a-be39-43af-bd53-d6ae3126c8bc,1: >$0 to <1K,202202,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0,0.0,0.0,0.000000,0.0,0.0,0.0
1084,f69bee69-087c-4906-a46c-0679aad7e0ea,3: $10K to <$100K,202202,0.000073,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0,0.0,0.0,112.800000,0.0,0.0,0.0
1085,f9309ef6-51d4-481b-81b4-ec6b1b41d542,3: $10K to <$100K,202202,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,11.105930,0,0.0,0.0,0.000000,0.0,0.0,0.0


###### Average spend by a customer when using both Service 1 and 2 ?

In [9]:
df.loc[df['Service 2'] > 0, 'Service 1'].mean()

628.2923321298974

In [11]:
df.describe()

,MONTHID,Service 1,Service 2,Service 3,Service 4,Service 5,Service 6,Service 7,Service 8,Service 9,...,Service 131,Service 132,Service 133,Service 134,Service 135,Service 136,Service 137,Service 138,Service 139,Service 140
count,1087.0,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,...,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000
mean,202202.0,17.918266,9.199632,1.149820,1.894442,0.073779,0.017316,2.063818,0.049276,0.174337,...,10.335193,19.221881,58.153516,0.004600,0.064829,0.695552,80.998178,6.308306,0.104147,0.566049
std,0.0,185.772580,151.444863,35.865864,56.196677,2.045170,0.570889,37.740835,1.139667,4.382415,...,156.159781,114.307785,1072.095230,0.151654,0.757357,11.718112,1550.652152,77.717138,2.002200,8.153280
min,202202.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,202202.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,202202.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,202202.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,202202.0,3836.887975,2500.000000,1180.515300,1847.439640,66.350168,18.822037,900.400000,30.477371,133.056000,...,4396.648032,1806.000000,34734.466159,5.000000,15.011406,324.000000,48465.775205,1479.650913,60.221250,245.799997


In [12]:
#Fill NaN values with 0
df.fillna(0,inplace=True)
df.shape

(1087, 143)

In [13]:
#Checking for duplicate accountID's
boolean = df['ACCOUNTID'].duplicated().any()
print("Is there a duplicate ID?",boolean)

Is there a duplicate ID? True


In [14]:
#Checking for duplicate accountID's
if df['ACCOUNTID'].duplicated().any():
    #Drop rows with duplicate ID
    df.drop_duplicates(subset ="ACCOUNTID",keep = 'first', inplace = True)

In [15]:
#df.drop(duplicateID)
df.shape

(1086, 143)

In [16]:
#Set AccountID as data frame index
df.set_index('ACCOUNTID', inplace=True)

###### Find the total number of services used by each customers?

In [17]:
#Drop Spend segment and MonthID columns
df.drop(df.columns[[0,1]], axis=1, inplace=True)
#Apply One-Hot-Encoding
def encode_services(x):
    if x == 0:
        return 0
    else:
        return 1
service_basket = df.applymap(encode_services)
#Total number of customers with unique accounts
total_accounts = len(service_basket.index)
print(total_accounts)
service_basket

1086


,Service 1,Service 2,Service 3,Service 4,Service 5,Service 6,Service 7,Service 8,Service 9,Service 10,...,Service 131,Service 132,Service 133,Service 134,Service 135,Service 136,Service 137,Service 138,Service 139,Service 140
ACCOUNTID,,,,,,,,,,,,,,,,,,,,,
076e4d78-2704-4463-bb18-326fdd51411d,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
0e9ebc97-7ea8-4fa8-b3b5-0c06d25fe63b,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
180f0ffb-2ee9-4b0a-925c-7db258e3fabd,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
18322cec-f95e-4e0d-ab23-938e74519211,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1c137916-8b2e-4dfd-8615-c66e6a1b4700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f2bd5d43-145f-4a72-8b73-1e36ea27ba15,1,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,1,0,0,0
f64dc09a-be39-43af-bd53-d6ae3126c8bc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f69bee69-087c-4906-a46c-0679aad7e0ea,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
#Group services by accounts and create a dictionary with list of services as values
service_dict = dict(
    list(
        service_basket.groupby(service_basket.index)
    )
)

# k: name of index, v: is a df
for k, v in service_dict.items():               
    check = v.columns[(v == 1).any()]
    if len(check) > 0:
        #print((k, check.to_list()))
        print(k,len(check.to_list()))

076e4d78-2704-4463-bb18-326fdd51411d 31
0e9ebc97-7ea8-4fa8-b3b5-0c06d25fe63b 28
180f0ffb-2ee9-4b0a-925c-7db258e3fabd 31
18322cec-f95e-4e0d-ab23-938e74519211 34
1c137916-8b2e-4dfd-8615-c66e6a1b4700 2
30d3c447-6fcd-4adf-9593-c4edbaff57ee 33
32fef345-b5f4-42d1-b334-b9341b402eac 24
339575cc-806f-4d15-b71e-7024ff7debb9 33
34f431f3-b1f6-4d8c-acad-636b7df3978f 9
3f3c677e-e87c-4d1b-98ef-d9f68bae53ae 36
4411a87e-63e6-4096-b4dd-645b0a6fcb20 38
4526a920-2791-4069-b6b5-b99716de9f70 11
4ab3172e-aa6c-4b25-b6bb-65669c0a5f21 33
52c0a4b9-b14e-4b9b-bd33-eaa1780984af 21
54228618-5b08-41b3-9b9e-e02b0ff5cc8d 5
584a6586-e260-4077-92c8-202e08846079 9
5bf07792-c5f5-4e47-a0de-3672f50b7eba 44
6202aa22-e295-4c75-b24c-5ca122c0a15b 10
652e94e5-9f55-4254-a2db-fd7e7ed503a7 14
669dcbf1-42dd-4a3e-aec2-1322ecf87fd1 59
70608ce5-2fb7-4ce7-accc-0d36a4764ca5 71
7268f4dc-c04d-41a6-ae7d-0ad6e637de5f 29
78763c59-d1df-479d-972b-730480fde1c6 32
78831da5-53bc-4439-b9b5-e11d4d25ca1f 24
79ffdbb7-7620-44b3-aacd-60ee809833dd 20
81cb

In [19]:
#Compute support
support = service_basket.mean()
support = pd.DataFrame(support, columns=['support']).sort_values('support',ascending=False)

# Print the support
support.head()

,support
Service 109,0.955801
Service 22,0.741252
Service 79,0.665746
Service 45,0.618785
Service 47,0.601289


In [20]:
support.describe()

,support
count,140.000000
mean,0.103341
std,0.177861
min,0.000921
25%,0.004604
50%,0.022560
75%,0.110267
max,0.955801


### Association rule mining

In [21]:
#Generating frequently brought service sets that have support value of atleast 5%(occurs 5 times out of 100 transactions)
frequent_service_set = apriori(service_basket,
                               min_support = 0.001,
                               max_len = 2, 
                               use_colnames = True)

frequent_service_set

,support,itemsets
0,0.203499,(Service 1)
1,0.003683,(Service 2)
2,0.001842,(Service 3)
3,0.019337,(Service 4)
4,0.004604,(Service 5)
...,...,...
5738,0.001842,"(Service 139, Service 136)"
5739,0.006446,"(Service 136, Service 140)"
5740,0.009208,"(Service 137, Service 138)"
5741,0.009208,"(Service 137, Service 140)"


In [22]:
#Generating rules
rules = association_rules(frequent_service_set, metric="confidence", min_threshold = 0.001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Service 2),(Service 1),0.003683,0.203499,0.001842,0.500000,2.457014,0.001092,1.593002
1,(Service 1),(Service 2),0.203499,0.003683,0.001842,0.009050,2.457014,0.001092,1.005416
2,(Service 4),(Service 1),0.019337,0.203499,0.017495,0.904762,4.446025,0.013560,8.363260
3,(Service 1),(Service 4),0.203499,0.019337,0.017495,0.085973,4.446025,0.013560,1.072904
4,(Service 5),(Service 1),0.004604,0.203499,0.004604,1.000000,4.914027,0.003667,inf
...,...,...,...,...,...,...,...,...,...
11225,(Service 138),(Service 137),0.080110,0.038674,0.009208,0.114943,2.972085,0.006110,1.086173
11226,(Service 137),(Service 140),0.038674,0.044199,0.009208,0.238095,5.386905,0.007499,1.254489
11227,(Service 140),(Service 137),0.044199,0.038674,0.009208,0.208333,5.386905,0.007499,1.214306
11228,(Service 138),(Service 140),0.080110,0.044199,0.010129,0.126437,2.860632,0.006588,1.094141


### Top 10 services used by cutomers?

In [32]:
rules.nlargest(10,'support')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2916,(Service 109),(Service 22),0.955801,0.741252,0.722836,0.756262,1.020249,0.014346,1.061581
2917,(Service 22),(Service 109),0.741252,0.955801,0.722836,0.975155,1.020249,0.014346,1.779006
8736,(Service 109),(Service 79),0.955801,0.665746,0.661142,0.691715,1.039007,0.024821,1.084237
8737,(Service 79),(Service 109),0.665746,0.955801,0.661142,0.993084,1.039007,0.024821,6.391160
2866,(Service 22),(Service 79),0.741252,0.665746,0.630755,0.850932,1.278163,0.137269,2.242288
2867,(Service 79),(Service 22),0.665746,0.741252,0.630755,0.947441,1.278163,0.137269,4.923015
5686,(Service 109),(Service 45),0.955801,0.618785,0.604972,0.632948,1.022889,0.013537,1.038587
5687,(Service 45),(Service 109),0.618785,0.955801,0.604972,0.977679,1.022889,0.013537,1.980110
6020,(Service 109),(Service 47),0.955801,0.601289,0.588398,0.615607,1.023812,0.013685,1.037248
6021,(Service 47),(Service 109),0.601289,0.955801,0.588398,0.978560,1.023812,0.013685,2.061563


Service 109 and Service 22 is used by most number of customers with a support of 72%

### Least used service among customers?

In [27]:
rules.iloc[rules['support'].idxmin()]

antecedents           (Service 2)
consequents           (Service 1)
antecedent support       0.003683
consequent support       0.203499
support                  0.001842
confidence                    0.5
lift                     2.457014
leverage                 0.001092
conviction               1.593002
Name: 0, dtype: object

Service 2 and Service 1 is used by least number of customers with a support of 0.1%

### Generate CSV report as output to the client

In [23]:
with open('output.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        #Column values
        header = ['Base Service','Attached Service', 'Attach Rate','Lift','Base Multiplier','Attach Multiplier',
                   'Base w/attach','Attach w/base','Base Freq','Attach Freq','Joint Freq','N', 'Base Accounts','Attach Accounts']
        # write the header to csv
        writer.writerow(header)
        #Row values
        for index, row in rules.iterrows():

            A = row['antecedents']
            B = row['consequents']
            #Attach Rate = Percentage of customers of base service that also purchase attached service
            #base_freq = (row['antecedent support'])
            lift  = row['lift']

            zip_object = zip(A, B)

            for i,j in zip_object:
                #Base Service
                base_service = i
                #Attach Service
                attached_service = j
                #N = Total number of accounts with the combination
                n = service_basket.loc[(service_basket[i] == 1) & (service_basket[j] == 1), i].count()
                #Attach Rate = Percentage of customers of base service that also purchase attached service
                base_accounts  = service_basket.loc[service_basket[i] == 1, i].count()
                attach_rate = n/base_accounts
                #Base freq = % of customers that use base service
                base_freq = base_accounts/total_accounts
                #Attach freq = % of customers that use attach service
                attach_accounts  = service_basket.loc[service_basket[j] == 1, j].count()
                attach_freq = attach_accounts/total_accounts
                #Joint freq = % of customers that use both service
                joint_freq = n/total_accounts
                
                
                #Base Multiplier = Average spend on A when also using B / Average spend on A when not using B
                avg_spend_AwB = df.loc[(df[j] > 0) & (df[i] > 0), i].mean()
                print(i,j,avg_spend_AwB)
                avg_spend_AwoB = df.loc[(df[j] == 0) & (df[i] > 0), i].mean()
                print(i,j,avg_spend_AwoB)
                                
                #Zero division error handling
                if (avg_spend_AwB == 0) or (avg_spend_AwoB == 0):
                    base_multiplier = 0
                else:
                    base_multiplier = avg_spend_AwB/avg_spend_AwoB
                    print(base_multiplier)
                #Attach multiplier  = Average spend on B when also using A / Average spend on B when not using A
                avg_spend_BwA = df.loc[(df[i] > 0) & (df[j] > 0), j].mean()
                avg_spend_BwoA = df.loc[(df[i] == 0) & (df[j] > 0), j].mean()
                #Zero division error handling
                if (avg_spend_BwA == 0) or (avg_spend_BwoA == 0):
                    attach_multiplier = 0
                else:
                    attach_multiplier = avg_spend_BwA/avg_spend_BwoA
                    
                #Base w/attach = Average spend on A when also using B
                total_spendA = df.loc[(df[i] > 0) & (df[j] > 0), i].sum()
                #Zero division error handling
                if n == 0:
                    basew_attach = 0
                else:
                    basew_attach = total_spendA/n
                                    
                #Attch w/base = Average spend on B when also using A
                total_spendB = df.loc[(df[i] > 0) & (df[j] > 0), j].sum()
                #Zero division error handling
                if n == 0:
                    attachw_base = 0
                else:
                    attachw_base = total_spendB/n

                #Output dataframe
                data = [base_service,attached_service,attach_rate,lift,
                         base_multiplier,attach_multiplier,basew_attach,
                         attachw_base,base_freq,attach_freq,
                         joint_freq,n,base_accounts,attach_accounts]

                # write the data to csv
                writer.writerow(data)
f.close()

Service 2 Service 1 2500.0
Service 2 Service 1 2500.0
1.0
Service 1 Service 2 1256.5846642597949
Service 1 Service 2 77.46112305222817
16.222133306956348
Service 4 Service 1 108.38180268433632
Service 4 Service 1 0.0021893221
49504.73148027708
Service 1 Service 4 112.95665780352053
Service 1 Service 4 85.79692464698354
1.3165583529746234
Service 5 Service 1 16.0394697505
Service 5 Service 1 nan
nan
Service 1 Service 5 332.814068315558
Service 1 Service 5 82.46798581194338
4.035675966119673
Service 7 Service 1 323.67761458334996
Service 7 Service 1 399.00374999999974
0.8112144674914714
Service 1 Service 7 147.24884236514998
Service 1 Service 7 87.5920438001245
1.681075540389899
Service 8 Service 1 8.927051775616667
Service 8 Service 1 0.000612151699999999
14583.071117202944
Service 1 Service 8 64.1008087945
Service 1 Service 8 88.80256011251424
0.721835144316597
Service 10 Service 1 797.0826903399995
Service 10 Service 1 462.72119574999965
1.722598181499016
Service 1 Service 10 275.5599